## Set up Coding Environment

In [66]:
# Import Necessary Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import sklearn
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import (confusion_matrix, 
                             roc_auc_score, 
                             average_precision_score)
from sklearn.model_selection import cross_val_score
import warnings
import time
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=UserWarning, module="distributed.utils_perf")
from multiprocessing import Pool
import dask.dataframe as dd
from dask.distributed import Client
from dask.diagnostics import ProgressBar
from collections import Counter

## Import Monthly Data

In [67]:
years = range(2011, 2020) # Avoid Recessions

In [68]:
df_data_dict = pd.read_excel('/Users/miracles/Desktop/practicum/Freddie Mac/READ_ME/SF LLD File Layout Release 36.xlsx', 
                             sheet_name='Monthly Performance Data File', 
                             header=1)

cols_to_del = ['Non MI Recoveries', 'Net Sales Proceeds', 
                'Delinquent Accrued Interest', 'Actual Loss Calculation',
                'Miscellaneous Expenses', 'Taxes and Insurance', 
                'Maintenance and Preservation Costs', 'Legal Costs', 
                'Expenses', 'MI Recoveries', 'Step Modification Flag', 
                'Modification Flag', 
                'Deferred Payment Plan', 'Modification Cost', 
                'Current Month Modification Cost', 'Borrower Assistance Status Code',
                'Due Date of Last Paid Installment (DDLPI)', 
                'Zero Balance Removal UPB', 
                'Zero Balance Effective Date', 
                'Current Deferred UPB',
                ]

dtypes = {
    'Loan Sequence Number': 'object',
    'Monthly Reporting Period': 'str',
    'Current Actual UPB': 'float64',
    'Current Loan Delinquency Status': 'object',
    'Loan Age': 'float32', # Changed to float due to potential NaN values
    'Remaining Months to Legal Maturity': 'float32', # Changed to float due to potential NaN values
    'Defect Settlement Date': 'str',
    'Modification Flag': 'object',
    'Zero Balance Code': 'float32',
    'Zero Balance Effective Date': 'str',
    'Current Interest Rate': 'float64',
    'Current Deferred UPB': 'float64',
    'Due Date of Last Paid Installment (DDLPI)': 'str',
    'MI Recoveries': 'float64',
    'Net Sales Proceeds': 'object', # 'Alpha-Numeric' but the range is not specified
    'Non MI Recoveries': 'float64',
    'Expenses': 'float64',
    'Legal Costs': 'float64',
    'Maintenance and Preservation Costs': 'float64',
    'Taxes and Insurance': 'float64',
    'Miscellaneous Expenses': 'float64',
    'Actual Loss Calculation': 'float64',
    'Modification Cost': 'float64',
    'Step Modification Flag': 'object',
    'Deferred Payment Plan': 'object',
    'Estimated Loan-to-Value (ELTV)': 'float32', # Changed to float due to potential NaN values
    'Zero Balance Removal UPB': 'float64',
    'Delinquent Accrued Interest': 'float64',
    'Delinquency Due to Disaster': 'object',
    'Borrower Assistance Status Code': 'object',
    'Current Month Modification Cost': 'float64',
    'Interest Bearing UPB': 'float64'
}

COLUMN_NAMES = df_data_dict['ATTRIBUTE NAME'].values
cols_to_read = list(set(COLUMN_NAMES) - set(cols_to_del))
column_indices = [list(COLUMN_NAMES).index(col) for col in cols_to_read]


In [69]:
# Initialization
paid_loans_indices = set()
charged_loans_indices = set()
aggregated_dataframes = []

data_path_template = '/Users/miracles/Desktop/practicum/Freddie Mac/Loan Level Data/historical_data_{0}Q{1}/historical_data_time_{0}Q{1}.txt'

# Start the overall timer
start_time_overall = time.time()

for year in years:
    for quarter in range(1, 5):
        # Start the timer for this specific quarter
        start_time = time.time()

        data_path = data_path_template.format(year, quarter)
        print('Processing ', year, 'Q', quarter)
        
        # Read only necessary columns
        monthly_data = pd.read_csv(data_path, 
                                   sep='|', 
                                   header=None, 
                                   names=df_data_dict['ATTRIBUTE NAME'].values,
                                   dtype=dtypes,
                                   usecols=column_indices)

        # Charged indices
        charged_loans_this_month = set(monthly_data.loc[monthly_data['Zero Balance Code'].isin([2.0, 3.0, 9.0]), 'Loan Sequence Number'].values)
        paid_loans_this_month = set(monthly_data.loc[monthly_data['Zero Balance Code'] == 1.0, 'Loan Sequence Number'].values)
        
        sampled_paid_indices = set(np.random.choice(list(paid_loans_this_month), len(charged_loans_this_month), replace=False))
        
        paid_loans_indices.update(sampled_paid_indices)
        charged_loans_indices.update(charged_loans_this_month)
        
        aggregated_dataframes.append(monthly_data[monthly_data['Loan Sequence Number'].isin(sampled_paid_indices.union(charged_loans_this_month))])
        
        del monthly_data

# Print overall processing time
print(f"\nTotal processing time: {time.time() - start_time_overall:.2f} seconds\n")

# Improved print statements
print(f"Total Paid Loans Indices: {len(paid_loans_indices)}")
print(f"Total Charged Loans Indices: {len(charged_loans_indices)}")
print(f"Total Selected Loan Indices: {len(paid_loans_indices) + len(charged_loans_indices)}")

df_monthly = pd.concat(aggregated_dataframes, ignore_index=True)


Processing  2011 Q 1
Processing  2011 Q 2
Processing  2011 Q 3
Processing  2011 Q 4
Processing  2012 Q 1
Processing  2012 Q 2
Processing  2012 Q 3
Processing  2012 Q 4
Processing  2013 Q 1
Processing  2013 Q 2
Processing  2013 Q 3
Processing  2013 Q 4
Processing  2014 Q 1
Processing  2014 Q 2
Processing  2014 Q 3
Processing  2014 Q 4
Processing  2015 Q 1
Processing  2015 Q 2
Processing  2015 Q 3
Processing  2015 Q 4
Processing  2016 Q 1
Processing  2016 Q 2
Processing  2016 Q 3
Processing  2016 Q 4
Processing  2017 Q 1
Processing  2017 Q 2
Processing  2017 Q 3
Processing  2017 Q 4
Processing  2018 Q 1
Processing  2018 Q 2
Processing  2018 Q 3
Processing  2018 Q 4
Processing  2019 Q 1
Processing  2019 Q 2
Processing  2019 Q 3
Processing  2019 Q 4

Total processing time: 699.86 seconds

Total Paid Loans Indices: 40116
Total Charged Loans Indices: 40116
Total Selected Loan Indices: 80232


## Import Original Data

In [70]:
## data dictionary
df_data_dict_orig = pd.read_excel('/Users/miracles/Desktop/practicum/Freddie Mac/READ_ME/SF LLD File Layout Release 36.xlsx', 
                             sheet_name='Origination Data File', 
                             header=1)

dtypes_orig = {
    'Credit Score': 'float64',
    'First Payment Date': 'str',
    'First Time Homebuyer Flag': 'object',
    'Maturity Date': 'str',
    'Metropolitan Statistical Area (MSA) Or Metropolitan Division': 'float64',
    'Mortgage Insurance Percentage (MI %)': 'float64',
    'Number of Units': 'float64',
    'Occupancy Status': 'object',
    'Original Combined Loan-to-Value (CLTV)': 'float64',
    'Original Debt-to-Income (DTI) Ratio': 'float64',
    'Original UPB': 'float64',
    'Original Loan-to-Value (LTV)': 'float64',
    'Original Interest Rate': 'float64',
    'Channel': 'object',
    'Prepayment Penalty Mortgage (PPM) Flag': 'object',
    'Amortization Type (Formerly Product Type)': 'object',
    'Property State': 'object',
    'Property Type': 'object',
    'Postal Code': 'float64',
    'Loan Sequence Number': 'object',
    'Loan Purpose': 'object',
    'Original Loan Term': 'float64',
    'Number of Borrowers': 'float64',
    'Seller Name': 'object',
    'Servicer Name': 'object',
    'Super Conforming Flag': 'object',
    'Pre-HARP Loan Sequence Number': 'object',
    'Program Indicator': 'object',
    'HARP Indicator': 'object',
    'Property Valuation Method': 'float64',
    'Interest Only (I/O) Indicator': 'object',
    'Mortgage Insurance Cancellation Indicator': 'object'
}

In [71]:
# Initialization
aggregated_dataframes_orig = []

data_path_template_orig = '/Users/miracles/Desktop/practicum/Freddie Mac/Loan Level Data/historical_data_{0}Q{1}/historical_data_{0}Q{1}.txt'

# Start the overall timer
start_time_overall = time.time()

for year in years:
    for quarter in range(1, 5):
        # Start the timer for this specific quarter
        start_time = time.time()

        data_path = data_path_template_orig.format(year, quarter)
        print(f'Processing {year}Q{quarter}')
        
        # Read only necessary columns
        orig_data = pd.read_csv(data_path, 
                                   sep='|', 
                                   header=None, 
                                   names=df_data_dict_orig['ATTRIBUTE NAME'].values,
                                   dtype=dtypes_orig)
        
        aggregated_dataframes_orig.append(pd.concat([
            orig_data[orig_data['Loan Sequence Number'].isin(paid_loans_indices)].reset_index(drop=True),
            orig_data[orig_data['Loan Sequence Number'].isin(charged_loans_indices)].reset_index(drop=True)
        ]))
        
        del orig_data

# Print overall processing time
print(f"\nTotal processing time: {time.time() - start_time_overall:.2f} seconds\n")

df_orig = pd.concat(aggregated_dataframes_orig, ignore_index=True)

Processing 2011Q1
Processing 2011Q2
Processing 2011Q3
Processing 2011Q4
Processing 2012Q1
Processing 2012Q2
Processing 2012Q3
Processing 2012Q4
Processing 2013Q1
Processing 2013Q2
Processing 2013Q3
Processing 2013Q4
Processing 2014Q1
Processing 2014Q2
Processing 2014Q3
Processing 2014Q4
Processing 2015Q1
Processing 2015Q2
Processing 2015Q3
Processing 2015Q4
Processing 2016Q1
Processing 2016Q2
Processing 2016Q3
Processing 2016Q4
Processing 2017Q1
Processing 2017Q2
Processing 2017Q3
Processing 2017Q4
Processing 2018Q1
Processing 2018Q2
Processing 2018Q3
Processing 2018Q4
Processing 2019Q1
Processing 2019Q2
Processing 2019Q3
Processing 2019Q4

Total processing time: 28.92 seconds



# Data Preprocessing

## Check Missing Rate

In [72]:
def missing_rate(df):
    """Returns the missing rate for each column in a dataframe."""
    return pd.DataFrame(df.isnull().sum() / len(df) * 100)

# Get missing rate for each dataframe
orig_missing_rate = missing_rate(df_orig).rename(columns={0: 'Column Name'})
# monthly_missing_rate = missing_rate(df_monthly).rename(columns={0: 'Column Name'})
orig_missing_rate

,Column Name
Credit Score,0.000000
First Payment Date,0.000000
First Time Homebuyer Flag,0.000000
Maturity Date,0.000000
Metropolitan Statistical Area (MSA) Or Metropolitan Division,26.495663
Mortgage Insurance Percentage (MI %),0.000000
Number of Units,0.000000
Occupancy Status,0.000000
Original Combined Loan-to-Value (CLTV),0.000000
Original Debt-to-Income (DTI) Ratio,0.000000


## Process original data

In [73]:
df_orig= df_orig[df_orig['Original Loan Term'] == 360]
df_orig = df_orig[df_orig['Credit Score'] != 9999]

# 1. Create new binary columns
# df_orig['Valid LTV'] = (df_orig['Original Loan-to-Value (LTV)'] != 999).astype(int)
df_orig['Valid DTI Ratio'] = (df_orig['Original Debt-to-Income (DTI) Ratio'] != 999).astype(int)

# Filter DataFrame using indices
df_charged_off = df_orig[df_orig['Loan Sequence Number'].isin(charged_loans_indices)]
df_paid_off = df_orig[df_orig['Loan Sequence Number'].isin(paid_loans_indices)]

# # 2. Print how many charged off and paid off loans are invalid
# print(f"Charged off loans with invalid Credit Score: {(df_charged_off['Valid Credit Score'] == 0).sum()}")
# print(f"Paid off loans with invalid Credit Score: {(df_paid_off['Valid Credit Score'] == 0).sum()}")

# print(f"Charged off loans with invalid LTV: {(df_charged_off['Valid LTV'] == 0).sum()}")
# print(f"Paid off loans with invalid LTV: {(df_paid_off['Valid LTV'] == 0).sum()}")

print(f"Charged off loans with invalid DTI Ratio: {(df_charged_off['Valid DTI Ratio'] == 0).sum()}")
print(f"Paid off loans with invalid DTI Ratio: {(df_paid_off['Valid DTI Ratio'] == 0).sum()}")

# Replace values based on conditions
df_orig['Super Conforming Flag'].replace({np.nan: '0', 'Y': '1'}, inplace=True)
df_orig['HARP Indicator'].replace({np.nan: '0', 'Y': '1'}, inplace=True)
df_orig['First Time Homebuyer Flag'].replace({'9':'Unknown'}, inplace=True)
df_orig['Program Indicator'].replace({np.nan:'Unknown'}, inplace=True)

# Handle missing values
df_orig.loc[df_orig['Mortgage Insurance Percentage (MI %)'] == 999, 'Mortgage Insurance Percentage (MI %)'] = np.nan
df_orig['Mortgage Insurance Percentage (MI %)'] = df_orig['Mortgage Insurance Percentage (MI %)'].fillna(df_orig['Mortgage Insurance Percentage (MI %)'].median())

# Drop specified columns
columns_to_drop = [
    'Metropolitan Statistical Area (MSA) Or Metropolitan Division', 
    'Prepayment Penalty Mortgage (PPM) Flag', 
    'Original Combined Loan-to-Value (CLTV)', 
    'Amortization Type (Formerly Product Type)', 
    'Interest Only (I/O) Indicator',
    'Pre-HARP Loan Sequence Number',
    'Seller Name',
    'Servicer Name'
]
df_orig.drop(columns=columns_to_drop, inplace=True)

# Impute missing values for 'Credit Score' where 'Valid Credit Score' is 0
# if 'Credit Score' in df_orig.columns:
#     mean_credit_score = df_orig.loc[df_orig['Valid Credit Score'] == 0, 'Credit Score'].mean()
#     df_orig.loc[df_orig['Valid Credit Score'] == 0, 'Credit Score'] = mean_credit_score

# # Impute missing values for 'Original Loan-to-Value (LTV)' where 'Valid LTV' is 0
# if 'Original Loan-to-Value (LTV)' in df_orig.columns:
#     mean_ltv = df_orig.loc[df_orig['Valid LTV'] == 0, 'Original Loan-to-Value (LTV)'].mean()
#     df_orig.loc[df_orig['Valid LTV'] == 0, 'Original Loan-to-Value (LTV)'] = mean_ltv

# Impute missing values for 'Original Debt-to-Income (DTI) Ratio' where 'Valid DTI Ratio' is 0
if 'Original Debt-to-Income (DTI) Ratio' in df_orig.columns:
    mean_dti_ratio = df_orig.loc[df_orig['Valid DTI Ratio'] == 0, 'Original Debt-to-Income (DTI) Ratio'].mean()
    df_orig.loc[df_orig['Valid DTI Ratio'] == 0, 'Original Debt-to-Income (DTI) Ratio'] = mean_dti_ratio


Charged off loans with invalid DTI Ratio: 24901
Paid off loans with invalid DTI Ratio: 5840


In [74]:
feature_to_keep = [
    ## loan
    'Loan Sequence Number',
    'First Payment Date',
    'Original Loan Term',
    'Original UPB',
    'Mortgage Insurance Percentage (MI %)',
    'Original Loan-to-Value (LTV)',
    'Original Interest Rate',
    'Channel',
    'Loan Purpose',
    'Super Conforming Flag',
    
    ## borrower
    'Credit Score',
    'First Time Homebuyer Flag',
    'Original Debt-to-Income (DTI) Ratio',
    'Number of Borrowers',
    
    ## property
    'Number of Units',
    'Occupancy Status',
    'Property State',
    'Property Type',
    'Property Valuation Method',
    
    ## Missing value indicator
#     'Valid Credit Score',
#     'Valid Original Loan Term',
#     'Valid LTV',
    'Valid DTI Ratio'
]

df_orig = df_orig.loc[:, feature_to_keep].reset_index(drop=True)

In [95]:
df_monthly

,Loan Sequence Number,Monthly Reporting Period,Current Actual UPB,Current Loan Delinquency Status,Loan Age,Remaining Months to Legal Maturity,Zero Balance Code,Current Interest Rate,Delinquency Due to Disaster,Interest Bearing UPB
0,F11Q10000044,201102,135000.0,0,0.0,360.0,NaN,4.625,0,135000.0
1,F11Q10000044,201103,135000.0,0,1.0,359.0,NaN,4.625,0,135000.0
2,F11Q10000044,201104,135000.0,0,2.0,358.0,NaN,4.625,0,135000.0
3,F11Q10000044,201105,134000.0,0,3.0,357.0,NaN,4.625,0,134000.0
4,F11Q10000044,201106,134000.0,0,4.0,356.0,NaN,4.625,0,134000.0
...,...,...,...,...,...,...,...,...,...,...
4191117,F19Q40540978,202210,159306.1,31,34.0,326.0,NaN,4.000,0,159306.1
4191118,F19Q40540978,202211,159306.1,32,35.0,325.0,NaN,4.000,0,159306.1
4191119,F19Q40540978,202212,159306.1,33,36.0,324.0,NaN,4.000,0,159306.1
4191120,F19Q40540978,202301,159306.1,34,37.0,323.0,NaN,4.000,0,159306.1


## Processing Monthly Data

In [76]:
month_missing_rate = missing_rate(df_monthly).rename(columns={0: 'Column Name'})

In [77]:
month_missing_rate

,Column Name
Loan Sequence Number,0.000000
Monthly Reporting Period,0.000000
Current Actual UPB,0.000000
Current Loan Delinquency Status,0.000000
Loan Age,0.000000
Remaining Months to Legal Maturity,0.000000
Defect Settlement Date,99.985660
Zero Balance Code,98.085668
Current Interest Rate,0.000000
Estimated Loan-to-Value (ELTV),67.900577


In [78]:
# Before filtering, count the initial number of rows and unique loans
initial_row_count = df_monthly.shape[0]
initial_loan_count = df_monthly['Loan Sequence Number'].nunique()

# Find all 'Loan Sequence Number' values with 'Current Loan Delinquency Status' equal to 'RA'
ra_loans = df_monthly[df_monthly['Current Loan Delinquency Status'] == 'RA']['Loan Sequence Number'].unique()

# Count how many rows and loans will be removed
rows_to_remove = df_monthly[df_monthly['Loan Sequence Number'].isin(ra_loans)].shape[0]
loans_to_remove = len(ra_loans)

# Then, filter out all rows with these 'Loan Sequence Numbers' from the DataFrame
df_monthly = df_monthly[~df_monthly['Loan Sequence Number'].isin(ra_loans)]

# After filtering, count the remaining number of rows and unique loans
final_row_count = df_monthly.shape[0]
final_loan_count = df_monthly['Loan Sequence Number'].nunique()

# Replace values based on conditions
df_monthly['Delinquency Due to Disaster'].replace({np.nan: 0}, inplace=True)
df_monthly.loc[df_monthly['Delinquency Due to Disaster'] == 'Y', 'Delinquency Due to Disaster'] = 1

# Replace '0' with integer 0 in 'Current Loan Delinquency Status'
df_monthly['Current Loan Delinquency Status'].replace({'0': 0}, inplace=True)

# Convert 'Current Loan Delinquency Status' to integer type
df_monthly['Current Loan Delinquency Status'] = df_monthly['Current Loan Delinquency Status'].astype(int)

# Drop specified columns
columns_to_drop_monthly = ['Estimated Loan-to-Value (ELTV)', 'Defect Settlement Date']
df_monthly.drop(columns=columns_to_drop_monthly, inplace=True)

# Output the results
print(f"Removed {rows_to_remove} rows associated with {loans_to_remove} loans due to 'RA' status.")
print(f"Initial rows: {initial_row_count}, Remaining rows: {final_row_count}")
print(f"Initial unique loans: {initial_loan_count}, Remaining unique loans: {final_loan_count}")

Removed 874038 rows associated with 16271 loans due to 'RA' status.
Initial rows: 4191122, Remaining rows: 3317084
Initial unique loans: 80232, Remaining unique loans: 63961


In [79]:
missing_rate(df_monthly).rename(columns={0: 'Column Name'})

,Column Name
Loan Sequence Number,0.00000
Monthly Reporting Period,0.00000
Current Actual UPB,0.00000
Current Loan Delinquency Status,0.00000
Loan Age,0.00000
Remaining Months to Legal Maturity,0.00000
Zero Balance Code,98.07177
Current Interest Rate,0.00000
Delinquency Due to Disaster,0.00000
Interest Bearing UPB,0.00000


In [80]:
# Identify the unique 'Loan Sequence Number' values present in both dataframes after the operations from the provided codes
loan_seq_nums_orig = df_orig['Loan Sequence Number'].unique()
loan_seq_nums_monthly = df_monthly['Loan Sequence Number'].unique()

# Find the common 'Loan Sequence Number' values between the two dataframes
common_loan_seq_nums = set(loan_seq_nums_orig) & set(loan_seq_nums_monthly)

# Filter both dataframes to retain only the rows with 'Loan Sequence Number' values present in both dataframes
df_orig = df_orig[df_orig['Loan Sequence Number'].isin(common_loan_seq_nums)]
df_monthly = df_monthly[df_monthly['Loan Sequence Number'].isin(common_loan_seq_nums)]

## Validation test for matching loan sequence number

In [81]:
def filter_and_stats_loans(df_orig, df_monthly):
    # Find all loans with 'Loan Age' starting from 0
    loans_with_zero_age = set(df_monthly[df_monthly['Loan Age'] == 0]['Loan Sequence Number'].unique())
    print(f"Loans with zero age: {len(loans_with_zero_age)}")

    # Find loans that have a non-NaN 'Zero Balance Code' at least once
    loans_with_ending = set(df_monthly.dropna(subset=['Zero Balance Code'])['Loan Sequence Number'].unique())
    print(f"Loans with a recorded ending: {len(loans_with_ending)}")

    # Filter the loans_with_zero_age to only include those that also have a recorded ending
    valid_loans = loans_with_zero_age & loans_with_ending

    # Filter df_monthly to only include loans with zero age and a recorded ending
    df_monthly_filtered = df_monthly[df_monthly['Loan Sequence Number'].isin(valid_loans)]

    # Check if ages are consecutive for each loan
    consecutively_valid_loans = set()
    for loan, group in df_monthly_filtered.groupby('Loan Sequence Number'):
        if list(group['Loan Age']) == list(range(len(group))):
            consecutively_valid_loans.add(loan)

    # Define valid loans as those with consecutive ages and a recorded ending
    valid_loans = valid_loans & consecutively_valid_loans

    # Define invalid loans
    invalid_loans = loans_with_zero_age - valid_loans
    
    # Remove invalid loans from dataframes
    df_orig_filtered = df_orig[df_orig['Loan Sequence Number'].isin(valid_loans)]
    df_monthly_filtered = df_monthly_filtered[df_monthly_filtered['Loan Sequence Number'].isin(valid_loans)]
    
    # Output statistics
    print(f"Total valid loans: {len(valid_loans)}")
    print(f"Total invalid loans: {len(invalid_loans)}")
    
    return df_orig_filtered, df_monthly_filtered

# Usage
df_orig_valid, df_monthly_valid = filter_and_stats_loans(df_orig, df_monthly)


Loans with zero age: 41966
Loans with a recorded ending: 46986
Total valid loans: 40741
Total invalid loans: 1225


In [89]:
# np.random.seed(42)
# rand_loan_idx = np.random.choice(np.array(list(selected_loan_idxes)), 1)[0]
rand_loan_idx = df_orig_valid.sample(1, random_state=42)['Loan Sequence Number'].values[0]
df_orig_valid.loc[df_orig_valid['Loan Sequence Number']==rand_loan_idx, :]


,Loan Sequence Number,First Payment Date,Original Loan Term,Original UPB,Mortgage Insurance Percentage (MI %),Original Loan-to-Value (LTV),Original Interest Rate,Channel,Loan Purpose,Super Conforming Flag,Credit Score,First Time Homebuyer Flag,Original Debt-to-Income (DTI) Ratio,Number of Borrowers,Number of Units,Occupancy Status,Property State,Property Type,Property Valuation Method,Valid DTI Ratio
55190,F16Q20262015,201608,360.0,144000.0,0.0,58.0,3.625,R,N,0,806.0,N,50.0,1.0,1.0,P,MA,CO,9.0,1


In [90]:
df_orig_valid.info(memory_usage=True)

<class 'pandas.core.frame.DataFrame'>
Index: 40741 entries, 0 to 60772
Data columns (total 20 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Loan Sequence Number                  40741 non-null  object 
 1   First Payment Date                    40741 non-null  object 
 2   Original Loan Term                    40741 non-null  float64
 3   Original UPB                          40741 non-null  float64
 4   Mortgage Insurance Percentage (MI %)  40741 non-null  float64
 5   Original Loan-to-Value (LTV)          40741 non-null  float64
 6   Original Interest Rate                40741 non-null  float64
 7   Channel                               40741 non-null  object 
 8   Loan Purpose                          40741 non-null  object 
 9   Super Conforming Flag                 40741 non-null  object 
 10  Credit Score                          40741 non-null  float64
 11  First Time Homebuyer

In [93]:
df_orig_valid.columns

Index(['Loan Sequence Number', 'First Payment Date', 'Original Loan Term',
       'Original UPB', 'Mortgage Insurance Percentage (MI %)',
       'Original Loan-to-Value (LTV)', 'Original Interest Rate', 'Channel',
       'Loan Purpose', 'Super Conforming Flag', 'Credit Score',
       'First Time Homebuyer Flag', 'Original Debt-to-Income (DTI) Ratio',
       'Number of Borrowers', 'Number of Units', 'Occupancy Status',
       'Property State', 'Property Type', 'Property Valuation Method',
       'Valid DTI Ratio'],
      dtype='object')

In [91]:
df_monthly_valid.info(memory_usage=True)

<class 'pandas.core.frame.DataFrame'>
Index: 2014608 entries, 0 to 4190990
Data columns (total 10 columns):
 #   Column                              Dtype  
---  ------                              -----  
 0   Loan Sequence Number                object 
 1   Monthly Reporting Period            object 
 2   Current Actual UPB                  float64
 3   Current Loan Delinquency Status     int64  
 4   Loan Age                            float32
 5   Remaining Months to Legal Maturity  float32
 6   Zero Balance Code                   float32
 7   Current Interest Rate               float64
 8   Delinquency Due to Disaster         object 
 9   Interest Bearing UPB                float64
dtypes: float32(3), float64(3), int64(1), object(3)
memory usage: 146.0+ MB


In [94]:
df_monthly_valid.columns

Index(['Loan Sequence Number', 'Monthly Reporting Period',
       'Current Actual UPB', 'Current Loan Delinquency Status', 'Loan Age',
       'Remaining Months to Legal Maturity', 'Zero Balance Code',
       'Current Interest Rate', 'Delinquency Due to Disaster',
       'Interest Bearing UPB'],
      dtype='object')

In [92]:
# Define paths for the CSV files on the desktop
orig_csv_path = "/Users/Miracles/Desktop/df_orig.csv"
monthly_csv_path = "/Users/Miracles/Desktop/df_monthly.csv"

# Output dataframes to CSV
df_orig_valid.to_csv(orig_csv_path, index=False)
df_monthly_valid.to_csv(monthly_csv_path, index=False)